In [1]:
import pandas as pd
pd.set_option('display.max_columns',100)
from flask import Flask, request

In [2]:
df_products = pd.read_json('./import_data.json')

In [3]:
# 각 제품의 주요 기능을 분석하여 함유된 영양소를 추정
def estimate_nutrients(product_description, keywords):
    nutrients = {key: 0 for key in keywords}
    for nutrient in nutrients.keys():
        if nutrient in product_description:
            nutrients[nutrient] += 10  # 임의의 값으로 설정
    return nutrients

# 각 제품의 추정된 영양소 함유량을 계산하여 부족한 영양소에 대한 점수 부여
def calculate_scores(df, user_requirements, keywords):
    scores = []
    for index, row in df.iterrows():
        product_name = row['prdlst_nm']
        product_description = row['iftkn_atnt_matr_cn'] + ' ' + row['primary_fnclty']
        estimated_nutrients = estimate_nutrients(product_description, keywords)
        
        # 부족한 영양소에 대한 점수 부여
        score = 0
        for nutrient, requirement in user_requirements.items():
            if estimated_nutrients.get(nutrient, 0) < requirement:
                score -= (requirement - estimated_nutrients.get(nutrient, 0))
        scores.append((product_name, score))
    return scores

# 부족한 영양소에 따라 제품 추천
def recommend_products(scores):
    recommended_products = sorted(scores, key=lambda x: x[1], reverse=True)[:5]  # 상위 5개 제품 추천
    return [product for product, _ in recommended_products]

# 사용자의 질환에 따라 영양소 섭취 요구량 정의
def get_user_requirements(disease_type):
    if disease_type == '위궤양':
        return {
            '비타민A': 1000,
            '비타민C': 1000,
            '비타민E': 1000,
            '아연': 1000,
            '식이섬유': 1000
        }
    elif disease_type == '만성 하부 호흡기 질환':
        return {
            '오메가': 1000,
            '비타민D': 1000,
            '비타민A': 1000,
            '아연': 1000,
            '마그네슘': 1000
        }
    elif disease_type == '당뇨':
        return {
            '오메가': 1000,
            '비타민D': 1000,
            '아연': 1000,
            '마그네슘': 1000
        }
    elif disease_type == '심장 질환':
        return {
            '비타민E': 1000,
            '비타민D': 1000,
            '비타민C': 1000,
            '비타민B': 1000,
            '아미노산': 1000,
            '아연': 1000
        }
    elif disease_type == '고혈압':
        return {
            '칼슘': 1000,
            '비타민D': 1000,
            '칼륨': 1000,
            '마그네슘': 1000
        }
    elif disease_type == '간질환' :
        return {
            '비타민E': 1000, 
            '비타민D': 1000,
            '비타민C' : 1000,
            '비타민B' : 1000,
            '아미노산' : 1000,
            '아연' : 1000
        }
    else:
        return None

# 질환에 따른 제품 추천
def recommend_products_for_disease(df_products, disease_type):
    if disease_type not in ['위궤양', '만성 하부 호흡기 질환', '당뇨', '심장 질환', '고혈압','간질환']:
        return "유효하지 않은 질환 유형입니다."
        
    else :
        # 사용자의 질환에 따른 영양소 요구량 설정
        user_requirements = get_user_requirements(disease_type)

        # 영양소 키워드 설정
        keywords = ['비타민A', '비타민C', '비타민E', '아연', '식이섬유', '오메가', '비타민D', '마그네슘', '비타민B', '아미노산', '칼슘', '칼륨']

        # 점수 계산 및 제품 추천
        scores = calculate_scores(df_products, user_requirements, keywords)
        recommended_products = recommend_products(scores)

        return recommended_products

In [4]:

# 위궤양 관련 제품 추천
recommend_products_for_disease(df_products, '위궤양 및 십이지장 질환')
# 만성 하부 호흡기 질환 관련 제품 추천
recommend_products_for_disease(df_products, '만성 하부 호흡기 질환')
# 당뇨 관련 제품 추천
recommend_products_for_disease(df_products, '당뇨')
# 심장 질환 관련 제품 추천
recommend_products_for_disease(df_products, '심장 질환')
# 고혈압 관련 제품 추천
recommend_products_for_disease(df_products, '고혈압')
# 간질환 관련 제품 추천
recommend_products_for_disease(df_products, '간질환')



['울트라비타민미네랄W', '닥터. 멀티비타민미네랄밸런스', '울트라비타민미네랄M', '포세이돈포맨', '멀티비타민미네랄 블루베리맛']

In [5]:
recommend_products_for_disease(df_products, '당뇨')

['인제칼슘포르테N비타민미네랄',
 '한빛인제칼슘포르테비타민미네랄',
 '진츄어블칼슘마그네슘아연 + 비타민D',
 '칼슘마그네슘아연+비타민C,D',
 '함소아 기린아 츄잉정']

In [ ]:
app = Flask(__name__)

@app.route('/')
def home():
    return "환영합니다! 질환 유형을 선택하여 관련 제품을 추천받으세요."
# 질환 관련 제품 추천 라우트
@app.route('/recommend/<disease_type>')
def recommend_products_for_disease_route(disease_type):
    global df_products
    df_products = pd.read_json('./import_data.json')
    # 질환 유형에 따른 제품 추천
    recommended_products = recommend_products_for_disease(df_products, disease_type)
    # 추천된 제품을 문자열 리스트로 반환
    recommendations = [f"{disease_type} 관련 추천된 제품:<br>"]
    for product in recommended_products:
        recommendations.append(f"{product}<br>")
    
    # 추천된 제품을 텍스트 형식으로 반환
    return '<br>'.join(recommendations)


if __name__ == '__main__':
    app.run(port=5005)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [06/Mar/2024 16:57:10] "GET /recommend/위궤양 HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2024 17:00:05] "GET /recommend/심장%20질환 HTTP/1.1" 200 -
